In [3]:
import xml.etree.ElementTree as ET 
import csv
xmlfile = '../IR_datasets/cs.stackexchange.com/PostLinks.xml'
tree = ET.parse(xmlfile)

In [5]:
root = tree.getroot()

In [6]:
def createCSV(root,postlinksFile):
    with open(postlinksFile,mode='w',encoding='utf-8') as postlinks:
        postlinks = csv.writer(postlinks, delimiter=',')
        postlinks.writerow(['Id','PostId','RelatedPostId','LinkTypeId'])
        for row in root.findall('row'):
            Id = row.get('Id')
            postId = row.get('PostId')
            relatedpostId = row.get('RelatedPostId')
            linktypeId = row.get('LinkTypeId')
            postlinks.writerow([Id,postId,relatedpostId,linktypeId])


In [7]:
createCSV(root,"../csv_files/postlinks.csv")

In [8]:
import pandas as pd
postlinksDF = pd.read_csv('../csv_files/postlinks.csv')

In [9]:
type(postlinksDF)

pandas.core.frame.DataFrame

In [10]:
postlinksDF

,Id,PostId,RelatedPostId,LinkTypeId
0,865,155,143,1
1,891,143,155,1
2,1139,192,57,1
3,1789,245,256,1
4,2699,341,302,1
5,3084,390,110,1
6,3114,393,110,1
7,3140,394,110,1
8,3191,401,367,1
9,3302,396,311,1


In [11]:
postlinksDF.shape

(9855, 4)

In [12]:
#final list of dict to keep track of related post ids and answer ids for each question id
postLinks = []
#to keep track of questions that were already encountered
posts = [] 

for index, row in postlinksDF.iterrows():
    temp = {'postID': None,'relatedID': [],'answerID': [] } 
    temp['postID'] = row['PostId']
    if(row['PostId'] in posts and len(posts)!=0):
        for i in range(0,len(posts)):
            if(row['PostId'] == postLinks[i]['postID']):
                postLinks[i]['relatedID'].append(row["RelatedPostId"])
                postLinks[i]['answerID'].append(row["Id"])
    else:  
        temp['relatedID'].append(row["RelatedPostId"])
        temp['answerID'].append(row["Id"])   
        posts.append(temp['postID'])
        postLinks.append(temp)       


In [13]:
len(postLinks)

6579

In [14]:
import pickle

#load the file of recommendations made by our model for each question
recommended = pickle.load(open("../models/ques_recommendations.dat","rb"))


In [15]:
#Function to evaluate recommendations for a question
#Evaluation made against the questions present in PostLinks.xml
#Weighted average of the number of questions from relatedID list are present in the list of recommendations 
#and the number of questions from relatedID list that are present in the top half of the list of recommendations

def evaluate():
    score  = 0
    for i in range(0,len(postLinks)):
        present = 0 
        top = 0
        count = len(postLinks[i]['relatedID'])
        if postLinks[i]['postID'] in recommended.keys():
            idx = 0
            for rec in recommended[postLinks[i]['postID']]:
                idx+=1
                if rec[0] in postLinks[i]['relatedID']:
                    present+=1
                    if (idx >= int(len(postLinks[i]['relatedID'])/2)):
                        top+=1
            if(present):
                score+= (0.3* (present/count)+ 0.7*(top/present))
    return (score/len(postLinks))
    

In [17]:
evaluate()

0.01878958301666918